In [ ]:
import cv2
import numpy as np
import time
from skimage.feature import hog

def extract_frames_from_video(video_path, take_every_n_frame = 50):
    # Открываем видеофайл
    cap = cv2.VideoCapture(video_path)
    
    # Проверяем, удалось ли открыть видеофайл
    if not cap.isOpened():
        print(f"Ошибка: не удалось открыть видеофайл {video_path}")
        return []

    frames = []
    c = 0
    
    while cap.isOpened():
        # Читаем кадр за кадром
        ret, frame = cap.read()
        c+=1
        
        if not ret:
            break
        
        # Добавляем кадр в список
        if c%take_every_n_frame == 0: frames.append(frame)
    
    # Освобождаем захват видео
    cap.release()
    
    return np.array(frames)

def get_the_most_complicated_frames(video_path, take_every_n_frame = 50):
    frames = extract_frames_from_video(video_path, take_every_n_frame)
    print(f"Извлечено {len(frames)} кадров из видео")

    average_hogs = []
    for i in range(len(frames)):
        # Загрузка изображения
        image = cv2.cvtColor(frames[i],cv2.COLOR_BGR2GRAY)
        
        #Вычисление HOG
        fd = hog(image, pixels_per_cell=(16, 16), cells_per_block=(4, 4))
        # Вычисление среднего значения HOG дескрипторов
        average_hog_descriptor = np.mean(fd) #выше коэффициент - больше деталей и обьектов в изображении
        average_hogs.append(average_hog_descriptor)
        
    entropies = []
    for input_image in frames:
        # Загрузка изображения
        image = cv2.cvtColor(input_image,cv2.COLOR_RGB2GRAY)
       
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        # Нормализация гистограммы
        hist = hist / np.sum(hist)
        
        # Вычисление меры энтропии
        entropy = -np.sum(hist * np.log2(hist + 1e-10))
        entropies.append(entropy)

    entropies_ = entropies-np.mean(entropies) 

    k = 0.1

    more_detailed_than_usual_frame_indexes = np.where((entropies_ > np.mean(entropies_)+k*np.std(entropies_)) & 
                                                      (average_hogs > np.mean(average_hogs)+k*np.std(average_hogs)))[0]
    less_detailed_than_usual_frame_indexes = np.where((entropies_ < np.mean(entropies_)-k*np.std(entropies_)) & 
                                                      (average_hogs < np.mean(average_hogs)-k*np.std(average_hogs)))[0]
    
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    mspf = 1000/fps #milliseconds per frame

    coef = []

    for ind in list(more_detailed_than_usual_frame_indexes):
        coef.append([ ( round((abs(int(ind)-1)*mspf)), round((int(ind)+1)*mspf) ), abs(float(entropies_[ind]+ average_hogs[ind])) ])

    for ind in list(less_detailed_than_usual_frame_indexes):
        coef.append([ ( round((abs(int(ind)-1)*mspf)), round((int(ind)+1)*mspf) ), -1*abs(float(entropies_[ind]+ average_hogs[ind])) ])
    
    coef.sort(key=lambda x: x[0][0])
    
    return coef

    

# Пример использования
start_time = time.time()

video_path = '/Users/polinanazarova/Desktop/2ef3ec7442459323bfb283992ffe001f.mp4'
coml_coef = get_the_most_complicated_frames(video_path)

end_time = time.time()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time} секунд")